In [1]:
import pandas as pd
import ujson as json
from datetime import datetime, timedelta
from typing import Dict, List, Literal, Optional, TypeAlias, Tuple
from concurrent.futures import ThreadPoolExecutor, as_completed
from script import FedInvestFetcher
from runner import bcolors

import nest_asyncio
nest_asyncio.apply()

%load_ext autoreload
%autoreload 2

In [16]:
def auction_df_filterer(historical_auctions_df: pd.DataFrame):
    historical_auctions_df["issue_date"] = pd.to_datetime(
        historical_auctions_df["issue_date"]
    )
    historical_auctions_df["maturity_date"] = pd.to_datetime(
        historical_auctions_df["maturity_date"]
    )
    historical_auctions_df["auction_date"] = pd.to_datetime(
        historical_auctions_df["auction_date"]
    )

    historical_auctions_df.loc[
        historical_auctions_df["original_security_term"].str.contains(
            "29-Year", case=False, na=False
        ),
        "original_security_term",
    ] = "30-Year"
    historical_auctions_df.loc[
        historical_auctions_df["original_security_term"].str.contains(
            "30-", case=False, na=False
        ),
        "original_security_term",
    ] = "30-Year"

    historical_auctions_df = historical_auctions_df[
        (historical_auctions_df["security_type"] == "Bill")
        | (historical_auctions_df["security_type"] == "Note")
        | (historical_auctions_df["security_type"] == "Bond")
    ]

    return historical_auctions_df

def process_cusip_otr_daterange(cusip, historical_auctions_df, date_column):
    try:
        tenor = historical_auctions_df[historical_auctions_df["cusip"] == cusip][
            "original_security_term"
        ].iloc[0]
        tenor_df: pd.DataFrame = historical_auctions_df[
            historical_auctions_df["original_security_term"] == tenor
        ].reset_index()
        otr_df = tenor_df[tenor_df["cusip"] == cusip]
        otr_index = otr_df.index[0]
        start_date: pd.Timestamp = otr_df[date_column].iloc[0]
        start_date = start_date.to_pydatetime()

        if otr_index == 0:
            return cusip, {"start_date": start_date, "end_date": datetime.today().date()}

        if otr_index < len(tenor_df) - 1:
            end_date: pd.Timestamp = tenor_df[date_column].iloc[otr_index - 1]
            end_date = end_date.to_pydatetime()
        else:
            end_date = datetime.today().date()
        
        print(bcolors.OKGREEN + f"Mapped {cusip}" + bcolors.ENDC)
        return cusip, {"start_date": start_date, "end_date": end_date}
    except Exception as e:
        print(bcolors.FAIL + f"Couldnt map {cusip} - {str(e)}" + bcolors.ENDC)
        return cusip, {"start_date": None, "end_date": None}


def get_otr_date_ranges(
    historical_auctions_df: pd.DataFrame, cusips: List[str], use_issue_date: bool = True
) -> Dict[str, Tuple[datetime, datetime]]:

    historical_auctions_df = auction_df_filterer(historical_auctions_df)
    date_column = "issue_date" if use_issue_date else "auction_date"
    historical_auctions_df = historical_auctions_df.sort_values(
        by=[date_column], ascending=False
    )
    historical_auctions_df = historical_auctions_df[
        historical_auctions_df["issue_date"].dt.date < datetime.today().date()
    ]
    historical_auctions_df = historical_auctions_df.drop_duplicates(
        subset=["cusip"], keep="last"
    )

    cusip_daterange_map = {}
    with ThreadPoolExecutor() as executor:
        futures = {
            executor.submit(
                process_cusip_otr_daterange, cusip, historical_auctions_df, date_column
            ): cusip
            for cusip in cusips
        }

        for future in as_completed(futures):
            cusip, date_range = future.result()
            cusip_daterange_map[cusip] = date_range

    return cusip_daterange_map

In [17]:
def write_nested_dict_to_json(data: Dict[str, Dict[str, datetime]], file_path: str):
    json_ready_data = {}
    for cusip, dates in data.items():
        try:
            json_ready_data[cusip] = {
                "start_date": (
                    dates["start_date"].strftime("%Y-%m-%d")
                    if dates["start_date"]
                    else None
                ),
                "end_date": (
                    dates["end_date"].strftime("%Y-%m-%d")
                    if dates["end_date"]
                    else None
                ),
            }
            print(bcolors.OKGREEN + f"Mapped {cusip} to JSON" + bcolors.ENDC)
        except Exception as e:
            print(
                bcolors.FAIL + f"Couldnt map {cusip} to JSON- {str(e)}" + bcolors.ENDC
            )

    with open(file_path, "w") as json_file:
        json.dump(json_ready_data, json_file, indent=4)

In [18]:
auction_df = fedinvest_fetcher = FedInvestFetcher(use_ust_issue_date=True).get_auctions_df()
auction_df = auction_df_filterer(auction_df)
auction_df = auction_df[auction_df["issue_date"].dt.date < datetime.today().date()]
auction_df

,cusip,security_type,auction_date,issue_date,maturity_date,price_per100,allocation_pctage,avg_med_yield,bid_to_cover_ratio,comp_accepted,...,security_term,original_security_term,security_term_week_year,primary_dealer_accepted,primary_dealer_tendered,reopening,total_accepted,total_tendered,treas_retail_accepted,treas_retail_tenders_accepted
191,912797MK0,Bill,2024-08-12,2024-08-15,2025-02-13,97.575861,36.280000,null,2.730000,67602581000,...,26-Week,26-Week,26-Week,25933500000,131875000000,No,76404564800,197600562800,1107659100,Yes
190,912797LD7,Bill,2024-08-12,2024-08-15,2024-11-14,98.718417,71.460000,null,2.950000,73407378200,...,13-Week,26-Week,13-Week,23220150000,133630000000,Yes,82953680400,231020031200,1255200800,Yes
187,912810UC0,Bond,2024-08-08,2024-08-15,2054-08-15,98.928757,14.970000,4.225000,2.310000,24941455500,...,30-Year,30-Year,30-Year,4784470000,32199000000,No,29754413000,62379781500,21843600,Yes
186,912797LG0,Bill,2024-08-08,2024-08-13,2024-09-10,99.588944,24.470000,null,2.770000,88633890500,...,4-Week,17-Week,4-Week,23949605000,162210000000,Yes,95294150400,263150971000,5176387700,Yes
185,912797LS4,Bill,2024-08-08,2024-08-13,2024-10-08,99.192667,12.700000,null,2.740000,88360476500,...,8-Week,17-Week,8-Week,25152425000,160225000000,Yes,90278113100,246797302600,1135964500,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,9127935W2,Bill,1980-01-02,1980-01-08,1981-01-02,null,3.000000,null,null,null,...,52-Week,52-Week,52-Week,null,null,No,4007825000,6329685000,null,null
2,9127933Y0,Bill,1979-12-28,1980-01-03,1980-04-03,null,21.000000,null,null,null,...,13-Week,null,13-Week,null,null,Yes,3257440000,5371540000,null,null
3,9127934U7,Bill,1979-12-28,1980-01-03,1980-07-03,null,94.000000,null,null,null,...,26-Week,26-Week,26-Week,null,null,No,3348725000,5954350000,null,null
1,912810CK2,Bond,1979-11-01,1979-11-15,2009-11-15,null,44.000000,10.440000,null,null,...,30-Year,30-Year,30-Year,null,null,No,2315000000,3594000000,null,null


In [19]:
otr_map = get_otr_date_ranges(auction_df, cusips=auction_df["cusip"].to_list(), use_issue_date=True)

Mapped 912797LG0
Mapped 912797LS4
Mapped 912797LD7
Mapped 912797LA3
Mapped 912797MJ3
Mapped 912797LC9
Mapped 912797LK1
Mapped 912797KZ9
Mapped 912797LZ8
Mapped 912797HE0
Mapped 912797LJ4
Mapped 912797JR9
Mapped 912797ME4
Mapped 912797MD6
Mapped 912797KY2
Mapped 912797KV8
Mapped 912797LH8
Mapped 912797KX4
Mapped 912797LG0
Mapped 912810UA4
Mapped 91282CKQ3
Mapped 912797KU0
Mapped 912797MC8
Mapped 912797LA3
Mapped 912797MB0
Mapped 912797LW5
Mapped 912797KZ9
Mapped 912797MA2
Mapped 912797LV7
Mapped 912797KW6
Mapped 912797KR7
Mapped 912797LY1
Mapped 912797GW1
Mapped 912797KT3
Mapped 912796ZV4
Mapped 912797LX3
Mapped 91282CKW0
Mapped 91282CKZ3
Mapped 912797LU9
Mapped 912797KQ9
Mapped 91282CKX8
Mapped 912797KY2
Mapped 912797LQ8
Mapped 912797LR6
Mapped 912797KP1
Mapped 912797KX4
Mapped 912797KM8
Mapped 91282CKY6
Mapped 912810UA4
Mapped 912797KL0
Mapped 91282CKQ3
Mapped 912797LS4
Mapped 91282CKV2
Mapped 912797GL5
Mapped 912797KH9
Mapped 912797KR7
Mapped 912797LT2
Mapped 912797KN6
Mapped 91282CK

In [21]:
write_nested_dict_to_json(data=otr_map, file_path="otr_date_range.json")

Mapped 912797LS4 to JSON
Mapped 912797LA3 to JSON
Mapped 912797KZ9 to JSON
Mapped 912797LG0 to JSON
Mapped 91282CLG4 to JSON
Mapped 912797KY2 to JSON
Mapped 912797MN4 to JSON
Mapped 912797JR9 to JSON
Mapped 912797LC9 to JSON
Mapped 912797MK0 to JSON
Mapped 912797LK1 to JSON
Mapped 912797KV8 to JSON
Mapped 912797LZ8 to JSON
Mapped 912797MJ3 to JSON
Mapped 912797LD7 to JSON
Mapped 912810UB2 to JSON
Mapped 912797MD6 to JSON
Mapped 912797KX4 to JSON
Mapped 91282CLF6 to JSON
Mapped 912797LH8 to JSON
Mapped 912797MG9 to JSON
Mapped 912797HE0 to JSON
Mapped 912810UC0 to JSON
Mapped 912797ME4 to JSON
Mapped 912797LJ4 to JSON
Mapped 91282CLD1 to JSON
Mapped 91282CLC3 to JSON
Mapped 912797MC8 to JSON
Mapped 912810UA4 to JSON
Mapped 91282CLB5 to JSON
Mapped 91282CKQ3 to JSON
Mapped 912797MB0 to JSON
Mapped 912797KU0 to JSON
Mapped 912797LW5 to JSON
Mapped 912797MA2 to JSON
Mapped 912797LV7 to JSON
Mapped 912797KW6 to JSON
Mapped 912797KR7 to JSON
Mapped 912797LY1 to JSON
Mapped 912797GW1 to JSON
